<a href="https://colab.research.google.com/github/Orikson/MAT-421/blob/main/MAT421_Module_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base-N and Binary Numbers

## Base-N
Python, as part of the standard library, has excellent support for conversions amongst the most commonly used bases in computer science, namely, base 2 (binary), base 8 (octal), and base 16 (hexadecimal).

In [7]:
n = 24 # @param {type:"integer"}

print('{0:d} in binary: {0:b}'.format(n, n))
print('{0:d} in octal: {0:o}'.format(n, n))
print('{0:d} in hexadecimal: {0:x}'.format(n,n))

24 in binary: 11000
24 in octal: 30
24 in hexadecimal: 18


Adding or multiplying numbers in binary is similar to adding or multiplying in base 10, but modulo 2 instead of modulo 10 on each digit. As expected, the end result has the same value whether you operate in base 10 or base 2

In [7]:
a = 20 # @param {type:"integer"}
b = 30 # @param {type:"integer"}
a_bin = bin(a)[2:]
b_bin = bin(b)[2:]

a_plus_b = int(a_bin,2) + int(b_bin,2)
a_times_b = int(a_bin,2) * int(b_bin,2)

print(f'a + b = {a + b}')
print(f'a * b = {a * b}')
print(f'a base 2 + b base 2 = {bin(a_plus_b)[2:]} base 2 = {a_plus_b} base 10')
print(f'a base 2 * b base 2 = {bin(a_times_b)[2:]} base 2 = {a_times_b} base 10')

a + b = 50
a * b = 600
a base 2 + b base 2 = 110010 base 2 = 50 base 10
a base 2 * b base 2 = 1001011000 base 2 = 600 base 10


What about converting an integer to an arbitrary base?

Consider the sum
$$n = \sum_0^i c_i \cdot b^i$$
Here, $c$ is a string of digits, and $i$ indicates the position along the string from right to left. $b$ is the base with respect to the string of digits.

This sum describes any number with respect to its digits. Notably, the least significant digit, where $i=0$, contributes the term $c_0 \cdot b^0 = c_0$. Taking this term out, and factoring out the base once, the sum becomes
$$n = b \left(\sum_1^i c_i \cdot b^{i-1} \right) + c_0$$ When dividing $n$ by $b$, we find that the remainder is simply $c_0$, the least significant digit. As such, to find the least significant digit, we simply have to compute $c_0 = n \pmod b$

We can repeat this process to find the remaining digits $c_i$ by subtracting the last digit we found from $n$, and dividing by a factor of $b$. For example, to find $c_1$, we note that
$$\frac{n - c_0}{b} = b \left( \sum_2^i c_i \cdot b^{i-2} \right) + c_1$$
and can compute $c_1 = \frac{n - c_0}{b} \pmod b$.

In [25]:
def to_base(n, b):
  if n == 0:
    return []
  c0 = n % b
  return to_base(n // b, b) + [c0]

n = 15326 # @param {type:"integer"}
b = 12 # @param {type:"integer"}

digits = to_base(n, b)

print(f'{n} in base {b} has digits {digits}')

s = 0
t = ''
for i, d in enumerate(digits[::-1]):
  s += d * pow(b, i)
  t += f' + {d} * {b}^{i}'
t = t[3:]

print(f'{t} = {s}')

15326 in base 12 has digits [8, 10, 5, 2]
2 * 12^0 + 5 * 12^1 + 10 * 12^2 + 8 * 12^3 = 15326


# Floating Point Numbers

In [1]:
import sys
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

Given the above sys information, we find that python floats follow the IEEE754 double precision standard. This means that all floating point numbers are represented by 64 bits: 1 sign bit, 11 exponent bits (with a bias of 1023), and 52 fraction bits. All values then are represented as
$$n = (-1)^s 2^{e-1023} (1 + f)$$

For example, let's take the bit string `1 10000000010 1000000000000000000000000000000000000000000000000000` (example from the text).

First, how do we calculate the fractional part? We know we have a 52 bit fractional component, whose value is represented as
$$\sum_{i=1}^{52} c_i \frac{1}{2^i}$$
where $i$ is the 1-index in the bit string from left to right. This is equivalent to
$$\frac{1}{2^{52}} \sum_{i=1}^{52} c_i 2^{52-i}$$
which, notably, is just the bit string as an integer divided by $2^{52}$. Although the `int` type in python is typically 32 bits (signed or unsigned), for values with more bits than 32 default to use the `long` type, which uses 64 bits, and can store our 52 bit integer easily.

Here, we compute the value of the floating point from the given bit string. Note that accuracy is reduced at high precisions because python inherently uses floats for calculations.

In [10]:
def b_to_fraction(b):
  f = int(b,2)
  return f / pow(2,52)

def b_to_float(b):
  s = int(b[0],2)
  e = int(b[1:12],2)
  f = b_to_fraction(b[12:])
  return pow(-1, s) * pow(2, e-1023) * (1 + f)

b = '1100000000101000000000000000000000000000000000000000000000000000'
print(f'The bit string "{b}" represents the float {b_to_float(b)}')
print(f'Expected value, -12.0')

The bit string "1100000000101000000000000000000000000000000000000000000000000000" represents the float -12.0
Expected value, -12.0


How large is the difference between a given float and the next largest/smallest representable value?

To get the bits of a float, I am using the `struct` python package.

In [52]:
import struct
import numpy as np

n = 15.0 # @param {type:"number"}
b = bin(struct.unpack('>q', struct.pack('>d', n))[0])[2:]
t = '0' * (64 - len(b)) + b

def b_to_float(b):
  return struct.unpack('>d', int(b,2).to_bytes(8, 'big'))[0]

def b_largest_smallest(b):
  s = b[0]
  e = b[1:12]
  f = int(b[12:],2)

  f_s = bin(f - 1)[2:]
  f_l = bin(f + 1)[2:]

  r_s = b_to_float(s + e + f_s)
  r_l = b_to_float(s + e + f_l)
  return r_l, r_s

r_l, r_s = b_largest_smallest(t)
print('Next largest: {:.28f}'.format(r_l))
print('Next smallest: {:.28f}'.format(r_s))

print('Next largest according to numpy: {:.28f}'.format(n + np.spacing(n)))
print('Next smallest according to numpy: {:.28f}'.format(n - np.spacing(n)))

Next largest: 15.0000000000000017763568394003
Next smallest: 14.9999999999999982236431605997
Next largest according to numpy: 15.0000000000000017763568394003
Next smallest according to numpy: 14.9999999999999982236431605997


How about overflow? What happens if we try to exceed the maximum floating point numbers? Because we run out of bits, we are unable to represent floating point numbers outside the range of maximum, and are clipped to that range.

In [74]:
largest = (2**(2046-1023))*((1 + sum(0.5**np.arange(1, 53))))
smallest = (2**(1-1023))*(1+0)
print(f'largest is sys.float_info.max? {largest == sys.float_info.max}')
print(f'largest + 1 is sys.float_info.max? {largest + 1 == sys.float_info.max}')
print(f'smallest is sys.float_info.min? {smallest == sys.float_info.min}')

largest is sys.float_info.max? True
largest + 1 is sys.float_info.max? True
smallest is sys.float_info.min? True


# Round-off Errors

Because of imprecision and gaps between floating point numbers, it is difficult to do absolute equivalences between floating points. Numbers may be rounded differently depending on the order of operations.

Take, for example, $0.1 + 0.2 + 0.3 = 0.6$, which is not true in pure python

In [76]:
print(f'0.1 + 0.2 + 0.3 = 0.6? {0.1 + 0.2 + 0.3 == 0.6}')
print(f'0.1 + 0.2 + 0.3 = {0.1 + 0.2 + 0.3}')

0.1 + 0.2 + 0.3 = 0.6? False
0.1 + 0.2 + 0.3 = 0.6000000000000001


Over time, and after many operations, this round-off error can accumulate and become much more substantial.

In [99]:
x = 15.0 # @param {type:"number"}
n = 1000 # @param {type:"integer"}

def sum_convolution(x, n):
  for _ in range(n):
    x += 1 / 17

  for _ in range(n):
    x -= 1 / 17

  return x

print('{x} after {n} sum iterations becomes {:.28f}'.format(sum_convolution(x, n // 10), x=x, n=n // 10))
print('{x} after {n} sum iterations becomes {:.28f}'.format(sum_convolution(x, n), x=x, n=n))
print('{x} after {n} sum iterations becomes {:.28f}'.format(sum_convolution(x, n * 10), x=x, n=n * 10))

def mult_convolution(x, n):
  for _ in range(n):
    x *= 1.1

  for _ in range(n):
    x /= 1.1

  return x

print('{x} after {n} mult iterations becomes {:.28f}'.format(mult_convolution(x, n // 10), x=x, n=n // 10))
print('{x} after {n} mult iterations becomes {:.28f}'.format(mult_convolution(x, n), x=x, n=n))

15.0 after 100 sum iterations becomes 15.0000000000000000000000000000
15.0 after 1000 sum iterations becomes 14.9999999999999928945726423990
15.0 after 10000 sum iterations becomes 14.9999999999999502620084967930
15.0 after 100 mult iterations becomes 14.9999999999999982236431605997
15.0 after 1000 mult iterations becomes 14.9999999999999893418589635985
